In [2]:
import pandas as pd
import numpy as np
df=pd.read_csv('Amazon_Unlocked_Mobile.csv')
df.head(10)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0
5,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,1,I already had a phone with problems... I know ...,1.0
6,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,The charging port was loose. I got that solder...,0.0
7,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,"Phone looks good but wouldn't stay charged, ha...",0.0
8,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I originally was using the Samsung S2 Galaxy f...,0.0
9,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,3,It's battery life is great. It's very responsi...,0.0


In [3]:
#Data cleaning and preparation
df.dropna(inplace=True)
df=df[df['Rating']!=3]
df['Positive Rated']=np.where(df['Rating']>3,1,0)

In [4]:
X=df['Reviews']
y=df['Positive Rated']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer().fit(X_train)
X_train[::2000]

97039     I bought a BB Black and was deliveried a White...
147913    The touch screen is messed up. It types or dia...
101961                                    excellent product
285082                                             Love it!
178889                                   nice for the price
                                ...                        
338895                                            very good
376706      had no problems with this may order another one
297379                      Slim, light and great mobility.
197443                                 Es un super telefono
283582    Fits great. Like the fact that the clip part c...
Name: Reviews, Length: 116, dtype: object

In [6]:
len(vect.get_feature_names())

53216

In [7]:
#Vectorising Training data
X_train_vectorized=vect.transform(X_train)
X_test_vectorized=vect.transform(X_test)
X_train_vectorized

<231207x53216 sparse matrix of type '<class 'numpy.int64'>'
	with 6117776 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.linear_model import LogisticRegression
log=LogisticRegression().fit(X_train_vectorized,y_train)
prediction=log.predict(X_test_vectorized)

C:\Users\Vallabh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [9]:
from sklearn.metrics import roc_auc_score
print('AUC: ', roc_auc_score(y_test, prediction))

AUC:  0.9205592121965233


In [10]:
feature_names=np.array(vect.get_feature_names())
feature_names

array(['00', '000', '0000', ..., '索尼大法好', '还不错', '좋군'], dtype='<U567')

In [11]:
sorted_coef_index=log.coef_[0].argsort()
print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))


Smallest Coefs:
['worst' 'garbage' 'junk' 'unusable' 'false' 'worthless' 'useless'
 'crashing' 'disappointing' 'awful']

Largest Coefs: 
['excelent' 'excelente' 'exelente' 'loving' 'loves' 'perfecto' 'excellent'
 'complaints' 'awesome' 'buen']


In [12]:
#tfdif
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5
vect = TfidfVectorizer(min_df=5).fit(X_train)
len(vect.get_feature_names())

17951

In [13]:
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

C:\Users\Vallabh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


AUC:  0.9265848398605042


In [14]:
sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['ciuld' 'equipmentthank' 'effectively' 'andsecond' 'ditches' 'custo'
 'encryption' 'conecction' 'blistering' 'anxious']

Largest Coefs: 
['cans' 'beining' 'atractive' 'compeition' '30000mah' '4months' 'compensa'
 'apps' '629' 'capabilites']


In [15]:
# These reviews are treated the same by our current model
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[0 0]


In [16]:
#ngrams to identify correctly using phrases in count vectoriser

vect=CountVectorizer(min_df=5,ngram_range=(1,2)).fit(X_train)
X_train[::2000]
#Vectorising Training data
X_train_vectorized=vect.transform(X_train)
X_test_vectorized=vect.transform(X_test)
X_train_vectorized

<231207x198917 sparse matrix of type '<class 'numpy.int64'>'
	with 12987648 stored elements in Compressed Sparse Row format>

In [17]:
from sklearn.linear_model import LogisticRegression
log=LogisticRegression().fit(X_train_vectorized,y_train)
prediction=log.predict(X_test_vectorized)
print('AUC: ', roc_auc_score(y_test, prediction))

C:\Users\Vallabh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


AUC:  0.9599601154447429


In [18]:
feature_names=np.array(vect.get_feature_names())
sorted_coef_index=log.coef_[0].argsort()
print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))


Smallest Coefs:
['no good' 'not happy' 'not worth' 'worst' 'junk' 'not satisfied'
 'garbage' 'not good' 'terrible' 'defective']

Largest Coefs: 
['excelent' 'excelente' 'not bad' 'excellent' 'exelente' 'perfect'
 'awesome' 'no problems' 'no issues' 'perfecto']


In [19]:
print(log.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[1 0]
